# A/B Test: 광고 문구 생성 비교

## 목적
- **대조군 A**: 일반적인 OpenAI API를 활용해 단순히 생성한 광고 문구
- **실험군 B**: 업종별 분류(18개 하위 그룹)를 통해 특화된 광고 문구

## 가설
업종별 특화 광고 문구 시스템(B)이 일반 광고 문구 생성(A)보다 더 업종 특성에 맞고 효과적인 광고 문구를 생성한다.

## 테스트 구성
- 18개 하위 그룹별 10개의 한글 예시 문장
- 총 180개의 테스트 케이스
- 4개의 셀로 구성: S등급, A등급, B+C등급, D+E등급

## 0. 환경 설정 및 모듈 로드

In [1]:
import os
import sys
import json
from typing import Dict, List
from dotenv import load_dotenv
from openai import OpenAI

# 프로젝트 루트 경로 설정
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# 환경변수 로드
load_dotenv()

# 실험군 B용 모듈 import
from src.generation.text_generation import TextGenerator, PromptTemplateManager

print(f"프로젝트 경로: {PROJECT_ROOT}")
print("모듈 로드 완료!")

프로젝트 경로: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz
모듈 로드 완료!


## 1. A/B 테스트 함수 정의

In [2]:
# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 실험군 B용 텍스트 생성기 초기화
text_generator_b = TextGenerator(use_industry_config=True)


def generate_ad_copy_control_a(korean_input: str, max_length: int = 20) -> str:
    """
    대조군 A: 일반적인 OpenAI API를 사용한 광고 문구 생성
    
    특징:
    - 업종 분류 없이 단순 광고 문구 생성
    - 일반적인 광고 카피라이팅 지시문 사용
    - 실험군 B와 비슷한 길이로 생성
    """
    system_prompt = f"""당신은 광고 카피라이터입니다. 주어진 내용을 바탕으로 광고 문구를 작성하세요.

규칙:
1. {max_length}자 이내로 작성 (공백 포함)
2. 짧고 임팩트 있는 문구 작성
3. 광고 문구만 출력 (설명, 번호 등 제외)
4. 한국어로 작성
5. 특수문자, 이모지 사용 금지
"""
    
    user_prompt = f"""다음 내용으로 광고 문구를 만들어주세요:

{korean_input}

광고 문구:"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7,
            max_tokens=100
        )
        
        result = response.choices[0].message.content.strip()
        
        # 후처리
        result = result.replace('"', '').replace("'", "").replace('「', '').replace('」', '')
        result = result.replace("1. ", "").replace("- ", "")
        
        if len(result) > max_length:
            result = result[:max_length].strip()
        
        return result
        
    except Exception as e:
        return f"[Error] {str(e)}"


def generate_ad_copy_experiment_b(korean_input: str, max_length: int = 20) -> str:
    """
    실험군 B: 업종별 분류를 통한 특화 광고 문구 생성
    
    특징:
    - 한글 입력에서 자동 업종 감지
    - 18개 하위 그룹별 최적화된 톤앤매너 적용
    - 업종별 키워드, CTA, 타겟 고객 반영
    - AIDA 프레임워크 적용
    """
    try:
        result = text_generator_b.generate_ad_copy(
            user_input=korean_input,
            tone="auto",  # 업종에 맞는 톤 자동 선택
            max_length=max_length
        )
        return result
    except Exception as e:
        return f"[Error] {str(e)}"


def run_ab_test(test_cases: List[Dict], group_name: str):
    """
    A/B 테스트 실행 및 결과 출력
    
    Args:
        test_cases: 테스트 케이스 리스트 [{"group": "S-1", "input": "한글 입력"}]
        group_name: 그룹명 (예: "S등급")
    """
    print(f"\n{'='*100}")
    print(f"📊 {group_name} A/B 테스트 결과")
    print(f"{'='*100}\n")
    
    results = []
    
    for i, case in enumerate(test_cases, 1):
        group = case.get("group", "")
        korean_input = case.get("input", "")
        
        print(f"\n[{i}/{len(test_cases)}] {group}")
        print(f"-" * 80)
        
        # A/B 광고 문구 생성
        ad_copy_a = generate_ad_copy_control_a(korean_input)
        ad_copy_b = generate_ad_copy_experiment_b(korean_input)
        
        # 결과 출력
        print(f'한글 입력: "{korean_input}"')
        print(f'대조군 A: "{ad_copy_a}"')
        print(f'실험군 B: "{ad_copy_b}"')
        print(f"📏 길이 비교: A={len(ad_copy_a)}자, B={len(ad_copy_b)}자")
        
        results.append({
            "group": group,
            "korean_input": korean_input,
            "ad_copy_a": ad_copy_a,
            "ad_copy_b": ad_copy_b,
            "len_a": len(ad_copy_a),
            "len_b": len(ad_copy_b)
        })
    
    # 요약 통계
    avg_len_a = sum(r["len_a"] for r in results) / len(results)
    avg_len_b = sum(r["len_b"] for r in results) / len(results)
    
    print(f"\n\n{'='*100}")
    print(f"📈 {group_name} 요약 통계")
    print(f"{'='*100}")
    print(f"테스트 케이스 수: {len(results)}")
    print(f"대조군 A 평균 길이: {avg_len_a:.1f}자")
    print(f"실험군 B 평균 길이: {avg_len_b:.1f}자")
    
    return results

## 2. 테스트 데이터 정의

### 18개 하위 그룹별 10개의 한글 예시 문장

In [3]:
# ============================================
# S등급: 감성 및 미각 자극 그룹 (4개 그룹)
# ============================================

# S-1: 고온 조리 및 육즙 강조 (19개 업종)
S1_HOT_COOKING = [
    {"group": "S-1", "input": "삼겹살 맛집, 숯불에 직접 굽는 고기"},
    {"group": "S-1", "input": "치킨 전문점 야식 배달, 갓 튀긴 바삭함"},
    {"group": "S-1", "input": "피자 신메뉴 출시, 치즈가 듬뿍"},
    {"group": "S-1", "input": "마라탕 전문점, 얼얼한 매운맛 도전"},
    {"group": "S-1", "input": "스테이크 레스토랑, 특별한 날 코스 요리"},
    {"group": "S-1", "input": "중국집 짜장면, 정통 손맛 전수"},
    {"group": "S-1", "input": "수제버거 전문점, 육즙 가득 패티"},
    {"group": "S-1", "input": "곱창 전골 맛집, 회식 장소 추천"},
    {"group": "S-1", "input": "라멘 전문점, 진한 돈코츠 육수"},
    {"group": "S-1", "input": "생맥주 펍, 퇴근 후 한 잔의 여유"},
]

# S-2: 신선도 및 색감 강조 (8개 업종)
S2_FRESHNESS = [
    {"group": "S-2", "input": "횟집 신선회, 산지 직송 활어"},
    {"group": "S-2", "input": "초밥 오마카세, 장인의 손길"},
    {"group": "S-2", "input": "해산물 찜 전문, 게와 새우 가득"},
    {"group": "S-2", "input": "수산시장 직판, 싱싱한 자연산"},
    {"group": "S-2", "input": "복어 요리 전문, 담백한 복어탕"},
    {"group": "S-2", "input": "한우 전문 정육점, 1++ 등급 마블링"},
    {"group": "S-2", "input": "과일 전문점, 제철 딸기 특가"},
    {"group": "S-2", "input": "유기농 채소 가게, 건강한 식탁"},
    {"group": "S-2", "input": "연어 전문점, 노르웨이 직수입"},
    {"group": "S-2", "input": "젓갈 전문점, 전통 방식 숙성"},
]

# S-3: 감성 및 부드러운 질감 (21개 업종)
S3_EMOTIONAL = [
    {"group": "S-3", "input": "카페 신메뉴, 봄 시즌 딸기 라떼"},
    {"group": "S-3", "input": "베이커리 오픈, 갓 구운 소금빵"},
    {"group": "S-3", "input": "젤라또 아이스크림, 수제 파스텔 톤"},
    {"group": "S-3", "input": "전통 떡카페, 예쁜 화과자 세트"},
    {"group": "S-3", "input": "샌드위치 전문점, 건강한 아침 식사"},
    {"group": "S-3", "input": "북카페 오픈, 책과 함께하는 커피"},
    {"group": "S-3", "input": "화장품 매장, 봄 신제품 립스틱"},
    {"group": "S-3", "input": "건강기능식품, 면역력 증진 비타민"},
    {"group": "S-3", "input": "문구점 신상, 예쁜 다이어리 컬렉션"},
    {"group": "S-3", "input": "기념품샵, 여행 추억을 담은 소품"},
]

# S-4: 정갈함과 다채로움 (22개 업종)
S4_NEAT_VARIETY = [
    {"group": "S-4", "input": "한정식 맛집, 정갈한 반상 차림"},
    {"group": "S-4", "input": "김밥 전문점, 든든한 한 끼 식사"},
    {"group": "S-4", "input": "백반집 점심 특선, 엄마 손맛 정식"},
    {"group": "S-4", "input": "돼지국밥 전문, 뜨끈한 해장 한 그릇"},
    {"group": "S-4", "input": "냉면 전문점, 시원한 여름 별미"},
    {"group": "S-4", "input": "반찬 가게, 정성 담은 수제 반찬"},
    {"group": "S-4", "input": "분식집, 추억의 떡볶이와 순대"},
    {"group": "S-4", "input": "손칼국수 전문, 쫄깃한 면발"},
    {"group": "S-4", "input": "해물파전 맛집, 비 오는 날 막걸리"},
    {"group": "S-4", "input": "족발 보쌈 전문, 야식 배달 인기"},
]

# S등급 전체 테스트 케이스
S_GRADE_TEST_CASES = S1_HOT_COOKING + S2_FRESHNESS + S3_EMOTIONAL + S4_NEAT_VARIETY

print(f"S등급 테스트 케이스 수: {len(S_GRADE_TEST_CASES)}개")

S등급 테스트 케이스 수: 40개


In [4]:
# ============================================
# A등급: 비주얼 변화 및 자기관리 그룹 (4개 그룹)
# ============================================

# A-1: 인물 중심 뷰티 및 에스테틱 (5개 업종)
A1_BEAUTY = [
    {"group": "A-1", "input": "미용실 봄 스타일, 웨이브 펌 이벤트"},
    {"group": "A-1", "input": "피부관리실, 맑고 깨끗한 피부 케어"},
    {"group": "A-1", "input": "네일샵 신상 디자인, 봄꽃 네일 아트"},
    {"group": "A-1", "input": "성형외과 상담, 자연스러운 라인 교정"},
    {"group": "A-1", "input": "피부과 레이저 시술, 잡티 제거 상담"},
    {"group": "A-1", "input": "미용실 염색, 트렌디한 발레아주"},
    {"group": "A-1", "input": "에스테틱 페이셜, 광채 나는 피부"},
    {"group": "A-1", "input": "젤네일 전문, 고급스러운 글리터"},
    {"group": "A-1", "input": "남성 헤어샵, 깔끔한 투블럭 컷"},
    {"group": "A-1", "input": "속눈썹 연장, 자연스러운 볼륨"},
]

# A-2: 신체 변화 및 웰니스 (6개 업종)
A2_WELLNESS = [
    {"group": "A-2", "input": "헬스장 PT 프로그램, 1:1 맞춤 트레이닝"},
    {"group": "A-2", "input": "요가 스튜디오, 아침 명상 클래스"},
    {"group": "A-2", "input": "필라테스 센터, 체형 교정 프로그램"},
    {"group": "A-2", "input": "태권도 학원, 어린이 품새 교육"},
    {"group": "A-2", "input": "다이어트 클리닉, 3개월 체중 감량"},
    {"group": "A-2", "input": "마사지샵, 피로 회복 전신 케어"},
    {"group": "A-2", "input": "크로스핏 박스, 고강도 인터벌"},
    {"group": "A-2", "input": "복싱 체육관, 다이어트 복싱 입문"},
    {"group": "A-2", "input": "스피닝 센터, 신나는 그룹 운동"},
    {"group": "A-2", "input": "스파 웰니스, 아로마 테라피 힐링"},
]

# A-3: 패션 및 스타일 아이템 (23개 업종)
A3_FASHION = [
    {"group": "A-3", "input": "여성 의류 매장, 봄 신상 원피스"},
    {"group": "A-3", "input": "남성 정장 전문, 맞춤 수트 제작"},
    {"group": "A-3", "input": "아동복 가게, 귀여운 키즈 패션"},
    {"group": "A-3", "input": "신발 매장, 트렌디한 스니커즈"},
    {"group": "A-3", "input": "가방 브랜드, 가죽 토트백 신상"},
    {"group": "A-3", "input": "주얼리샵, 프리미엄 시계 컬렉션"},
    {"group": "A-3", "input": "안경점 신상, 트렌드 뿔테 안경"},
    {"group": "A-3", "input": "사진관, 가족 기념사진 촬영"},
    {"group": "A-3", "input": "웨딩샵, 우아한 레이스 드레스"},
    {"group": "A-3", "input": "한복 대여, 설날 전통 한복"},
]

# A-4: 섬세한 케어 및 감성 소품 (16개 업종)
A4_DELICATE_CARE = [
    {"group": "A-4", "input": "꽃집, 발렌타인 장미 꽃다발"},
    {"group": "A-4", "input": "반려동물 용품점, 강아지 봄 옷"},
    {"group": "A-4", "input": "동물병원, 반려견 건강검진 패키지"},
    {"group": "A-4", "input": "장난감 가게, 레고 신제품 출시"},
    {"group": "A-4", "input": "가구점, 미니멀 원목 책상"},
    {"group": "A-4", "input": "세탁소 서비스, 명품 가방 클리닝"},
    {"group": "A-4", "input": "피아노 학원, 어린이 레슨 모집"},
    {"group": "A-4", "input": "미술학원, 수채화 클래스 오픈"},
    {"group": "A-4", "input": "자전거 매장, 접이식 미니벨로"},
    {"group": "A-4", "input": "인테리어 소품샵, 북유럽 감성 조명"},
]

# A등급 전체 테스트 케이스
A_GRADE_TEST_CASES = A1_BEAUTY + A2_WELLNESS + A3_FASHION + A4_DELICATE_CARE

print(f"A등급 테스트 케이스 수: {len(A_GRADE_TEST_CASES)}개")

A등급 테스트 케이스 수: 40개


In [5]:
# ============================================
# B등급: 공간 체험 및 교육 그룹 (3개 그룹)
# ============================================

# B-1: 학습 및 몰입 (16개 업종)
B1_LEARNING = [
    {"group": "B-1", "input": "입시학원, 소수정예 수학 클래스"},
    {"group": "B-1", "input": "외국어 학원, 원어민 영어 회화"},
    {"group": "B-1", "input": "코딩 학원, 어린이 프로그래밍 교육"},
    {"group": "B-1", "input": "스터디카페, 조용하고 쾌적한 공간"},
    {"group": "B-1", "input": "운전학원, 친절한 1:1 교육"},
    {"group": "B-1", "input": "공무원 학원, 합격률 1위 커리큘럼"},
    {"group": "B-1", "input": "독서실 오픈, 24시간 개인 부스"},
    {"group": "B-1", "input": "유학원 상담, 미국 대학 컨설팅"},
    {"group": "B-1", "input": "취업 학원, 면접 스피치 특강"},
    {"group": "B-1", "input": "자격증 학원, 바리스타 과정 모집"},
]

# B-2: 여가 및 숙박 (19개 업종)
B2_LEISURE = [
    {"group": "B-2", "input": "호텔 프로모션, 오션뷰 스위트룸"},
    {"group": "B-2", "input": "펜션, 가족 단위 바베큐 독채"},
    {"group": "B-2", "input": "PC방 이벤트, 최신 사양 게이밍"},
    {"group": "B-2", "input": "노래방, 프리미엄 코인 노래방"},
    {"group": "B-2", "input": "글램핑장, 자연 속 럭셔리 캠핑"},
    {"group": "B-2", "input": "여행사, 제주도 3박 4일 패키지"},
    {"group": "B-2", "input": "찜질방, 힐링 스파 시설 안내"},
    {"group": "B-2", "input": "만화카페, 아늑한 독서 공간"},
    {"group": "B-2", "input": "보드게임 카페, 친구들과 함께"},
    {"group": "B-2", "input": "워터파크, 여름 물놀이 시즌"},
]

# B-3: 스포츠 및 활동 (10개 업종)
B3_SPORTS = [
    {"group": "B-3", "input": "수영장 회원 모집, 성인 강습 개강"},
    {"group": "B-3", "input": "골프 연습장, 실내 스크린 골프"},
    {"group": "B-3", "input": "볼링장 이벤트, 단체 대회 개최"},
    {"group": "B-3", "input": "당구장, 프로 레슨 프로그램"},
    {"group": "B-3", "input": "테니스장 오픈, 실내 코트 완비"},
    {"group": "B-3", "input": "탁구장, 동호인 리그전 개최"},
    {"group": "B-3", "input": "풋살장 대관, 최신 인조잔디"},
    {"group": "B-3", "input": "배드민턴장, 동호회 대관 안내"},
    {"group": "B-3", "input": "클라이밍 센터, 초보자 볼더링"},
    {"group": "B-3", "input": "스쿼시장, 1:1 레슨 프로그램"},
]

# ============================================
# C등급: 신뢰 기반 전문 서비스 그룹 (3개 그룹)
# ============================================

# C-1: 전문 사무 (18개 업종)
C1_PROFESSIONAL = [
    {"group": "C-1", "input": "부동산 중개, 역세권 신축 오피스텔"},
    {"group": "C-1", "input": "법무법인, 기업 법률 자문 서비스"},
    {"group": "C-1", "input": "세무사 사무소, 종합소득세 신고"},
    {"group": "C-1", "input": "회계사, 스타트업 재무 컨설팅"},
    {"group": "C-1", "input": "특허 사무소, 상표권 등록 대행"},
    {"group": "C-1", "input": "노무사, 근로계약서 검토 서비스"},
    {"group": "C-1", "input": "광고 대행사, 브랜드 마케팅 캠페인"},
    {"group": "C-1", "input": "경영 컨설팅, 중소기업 성장 전략"},
    {"group": "C-1", "input": "행정사, 인허가 대행 전문"},
    {"group": "C-1", "input": "번역 사무소, 법률 문서 번역"},
]

# C-2: 의료 및 케어 (16개 업종)
C2_MEDICAL = [
    {"group": "C-2", "input": "내과 의원, 건강검진 패키지 안내"},
    {"group": "C-2", "input": "치과 클리닉, 임플란트 상담 이벤트"},
    {"group": "C-2", "input": "한의원, 비만 한방 다이어트"},
    {"group": "C-2", "input": "정형외과, 척추 비수술 치료"},
    {"group": "C-2", "input": "안과 라식 센터, 시력교정 상담"},
    {"group": "C-2", "input": "이비인후과, 수면무호흡 검사"},
    {"group": "C-2", "input": "소아과, 영유아 예방접종 안내"},
    {"group": "C-2", "input": "산부인과, 임신 초기 검사"},
    {"group": "C-2", "input": "정신건강의학과, 스트레스 상담"},
    {"group": "C-2", "input": "요양원, 어르신 전문 케어 시설"},
]

# C-3: 기술 및 시공 (16개 업종)
C3_TECHNICAL = [
    {"group": "C-3", "input": "자동차 정비소, 엔진오일 교환 이벤트"},
    {"group": "C-3", "input": "세차장, 프리미엄 손세차 서비스"},
    {"group": "C-3", "input": "인테리어 업체, 아파트 리모델링"},
    {"group": "C-3", "input": "이사업체, 원룸 포장이사 서비스"},
    {"group": "C-3", "input": "에어컨 청소, 분해 완전 세척"},
    {"group": "C-3", "input": "방역 소독 업체, 바퀴벌레 퇴치"},
    {"group": "C-3", "input": "조경 관리, 아파트 정원 가꾸기"},
    {"group": "C-3", "input": "간판 제작, LED 채널 간판 시공"},
    {"group": "C-3", "input": "타일 시공, 욕실 리모델링 전문"},
    {"group": "C-3", "input": "도배 장판, 벽지 시공 견적 안내"},
]

# B+C등급 전체 테스트 케이스
BC_GRADE_TEST_CASES = B1_LEARNING + B2_LEISURE + B3_SPORTS + C1_PROFESSIONAL + C2_MEDICAL + C3_TECHNICAL

print(f"B+C등급 테스트 케이스 수: {len(BC_GRADE_TEST_CASES)}개")

B+C등급 테스트 케이스 수: 60개


In [6]:
# ============================================
# D등급: 목적형 소매 및 장비 그룹 (2개 그룹)
# ============================================

# D-1: 중장비 및 부품 (16개 업종)
D1_HEAVY_EQUIPMENT = [
    {"group": "D-1", "input": "타이어 전문점, 사계절 타이어 할인"},
    {"group": "D-1", "input": "자동차 부품점, 순정 브레이크 패드"},
    {"group": "D-1", "input": "철물점, 전동 공구 세트 특가"},
    {"group": "D-1", "input": "건축자재 판매, 시멘트 벽돌 도매"},
    {"group": "D-1", "input": "전기용품 가게, LED 조명 설치"},
    {"group": "D-1", "input": "중장비 대여, 굴삭기 렌탈 서비스"},
    {"group": "D-1", "input": "바닥재 전문, 마루 장판 시공"},
    {"group": "D-1", "input": "의료기기 판매, 혈압계 체온계 세트"},
    {"group": "D-1", "input": "중고 물품점, 가전제품 매입 판매"},
    {"group": "D-1", "input": "사료 전문점, 반려동물 사료 할인"},
]

# D-2: 일반 기기 (14개 업종)
D2_GENERAL_EQUIPMENT = [
    {"group": "D-2", "input": "핸드폰 매장, 최신 스마트폰 개통"},
    {"group": "D-2", "input": "컴퓨터 수리점, 노트북 SSD 업그레이드"},
    {"group": "D-2", "input": "가전제품 매장, 에어컨 여름 특가"},
    {"group": "D-2", "input": "복합기 임대, 사무실 프린터 렌탈"},
    {"group": "D-2", "input": "폰 수리 전문, 액정 교체 당일 수리"},
    {"group": "D-2", "input": "렌터카 업체, 장기 렌트 프로모션"},
    {"group": "D-2", "input": "인력사무소, 단기 아르바이트 모집"},
    {"group": "D-2", "input": "냉장고 수리, 컴프레서 교체 서비스"},
    {"group": "D-2", "input": "노트북 판매, 학생 할인 프로모션"},
    {"group": "D-2", "input": "TV 수리점, 대형 TV 출장 수리"},
]

# ============================================
# E등급: 인프라 및 특수 목적 그룹 (2개 그룹)
# ============================================

# E-1: 편의 및 인프라 (2개 업종)
E1_INFRASTRUCTURE = [
    {"group": "E-1", "input": "주유소, 리터당 100원 할인 이벤트"},
    {"group": "E-1", "input": "LPG 충전소, 자동차 가스 충전"},
    {"group": "E-1", "input": "셀프 주유소, 24시간 야간 운영"},
    {"group": "E-1", "input": "세차장 겸 주유소, 셀프 세차 시설"},
    {"group": "E-1", "input": "전기차 충전소, 급속 충전 스테이션"},
]

# E-2: 예절 및 의례 (2개 업종)
E2_CEREMONY = [
    {"group": "E-2", "input": "장례식장, 엄숙한 추모 공간 안내"},
    {"group": "E-2", "input": "납골당 분양, 평화로운 영면의 장소"},
    {"group": "E-2", "input": "상조 서비스, 장례 종합 패키지"},
    {"group": "E-2", "input": "수목장 안내, 자연과 함께하는 추모"},
    {"group": "E-2", "input": "추모관, 차분한 추모 시설 안내"},
]

# D+E등급 전체 테스트 케이스
DE_GRADE_TEST_CASES = D1_HEAVY_EQUIPMENT + D2_GENERAL_EQUIPMENT + E1_INFRASTRUCTURE + E2_CEREMONY

print(f"D+E등급 테스트 케이스 수: {len(DE_GRADE_TEST_CASES)}개")

D+E등급 테스트 케이스 수: 30개


## 3. A/B 테스트 실행

### 3.1 S등급 테스트 (감성 및 미각 자극)

In [7]:
# S등급 A/B 테스트 실행
s_grade_results = run_ab_test(S_GRADE_TEST_CASES, "S등급 (감성 및 미각 자극)")


📊 S등급 (감성 및 미각 자극) A/B 테스트 결과


[1/40] S-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 삼겹살 맛집, 숯불에 직접 굽는 고기
   감지된 업종: s1_hot_cooking
   추천 톤 적용: warm
   톤: warm, 최대 20자
✅ 생성 완료: 숯불 삼겹살 지금 주문!
한글 입력: "삼겹살 맛집, 숯불에 직접 굽는 고기"
대조군 A: "숯불 삼겹살의 진수!"
실험군 B: "숯불 삼겹살 지금 주문!"
📏 길이 비교: A=11자, B=13자

[2/40] S-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 치킨 전문점 야식 배달, 갓 튀긴 바삭함
   감지된 업종: s1_hot_cooking
   추천 톤 적용: warm
   톤: warm, 최대 20자
✅ 생성 완료: 갓 튀긴 바삭한 치킨
한글 입력: "치킨 전문점 야식 배달, 갓 튀긴 바삭함"
대조군 A: "바삭한 치킨, 지금 배달!"
실험군 B: "갓 튀긴 바삭한 치킨"
📏 길이 비교: A=14자, B=11자

[3/40] S-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 피자 신메뉴 출시, 치즈가 듬뿍
   감지된 업종: s1_hot_cooking
   추천 톤 적용: warm
   톤: warm, 최대 20자
✅ 생성 완료: 치즈 한가득, 지금 주문!
한글 입력: "피자 신메뉴 출시, 치즈가 듬뿍"
대조군 A: "치즈 폭발 피자 등장!"
실험군 B: "치즈 한가득, 지금 주문!"
📏 길이 비교: A=12자, B=14자

[4/40] S-1
--

### 3.2 A등급 테스트 (비주얼 변화 및 자기관리)

In [8]:
# A등급 A/B 테스트 실행
a_grade_results = run_ab_test(A_GRADE_TEST_CASES, "A등급 (비주얼 변화 및 자기관리)")


📊 A등급 (비주얼 변화 및 자기관리) A/B 테스트 결과


[1/40] A-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 미용실 봄 스타일, 웨이브 펌 이벤트
   감지된 업종: a1_beauty
   추천 톤 적용: friendly
   톤: friendly, 최대 20자
✅ 생성 완료: 봄, 나만의 웨이브로 변신!
한글 입력: "미용실 봄 스타일, 웨이브 펌 이벤트"
대조군 A: "봄, 웨이브로 변신하자!"
실험군 B: "봄, 나만의 웨이브로 변신!"
📏 길이 비교: A=13자, B=15자

[2/40] A-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 피부관리실, 맑고 깨끗한 피부 케어
   감지된 업종: a1_beauty
   추천 톤 적용: friendly
   톤: friendly, 최대 20자
✅ 생성 완료: 전문가의 맞춤 피부 변신
한글 입력: "피부관리실, 맑고 깨끗한 피부 케어"
대조군 A: "피부의 맑음을 되찾다"
실험군 B: "전문가의 맞춤 피부 변신"
📏 길이 비교: A=11자, B=13자

[3/40] A-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 네일샵 신상 디자인, 봄꽃 네일 아트
   감지된 업종: a1_beauty
   추천 톤 적용: friendly
   톤: friendly, 최대 20자
✅ 생성 완료: 봄꽃 네일로 변신해요!
한글 입력: "네일샵 신상 디자인, 봄꽃 네일 아트"
대조군 A: "봄꽃 네일로 새로움!"
실험군 B: "봄꽃 네일로 변신해요!"
📏 길이 비교: A=11자, B=12자


### 3.3 B+C등급 테스트 (공간 체험/교육 + 신뢰 기반 서비스)

In [9]:
# B+C등급 A/B 테스트 실행
bc_grade_results = run_ab_test(BC_GRADE_TEST_CASES, "B+C등급 (공간 체험/교육 + 신뢰 기반 서비스)")


📊 B+C등급 (공간 체험/교육 + 신뢰 기반 서비스) A/B 테스트 결과


[1/60] B-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 입시학원, 소수정예 수학 클래스
   감지된 업종: b1_learning
   추천 톤 적용: energetic
   톤: energetic, 최대 20자
✅ 생성 완료: 소수정예로 성적 UP! 지금 상담 신
한글 입력: "입시학원, 소수정예 수학 클래스"
대조군 A: "수학, 소수로 깊이 있게!"
실험군 B: "소수정예로 성적 UP! 지금 상담 신"
📏 길이 비교: A=14자, B=20자

[2/60] B-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 외국어 학원, 원어민 영어 회화
   감지된 업종: b1_learning
   추천 톤 적용: energetic
   톤: energetic, 최대 20자
✅ 생성 완료: 원어민과 함께하는 영어!
한글 입력: "외국어 학원, 원어민 영어 회화"
대조군 A: "원어민과의 자유로운 영어!"
실험군 B: "원어민과 함께하는 영어!"
📏 길이 비교: A=14자, B=13자

[3/60] B-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 코딩 학원, 어린이 프로그래밍 교육
   감지된 업종: b1_learning
   추천 톤 적용: energetic
   톤: energetic, 최대 20자
✅ 생성 완료: 코딩으로 미래를 여는 첫걸음!
한글 입력: "코딩 학원, 어린이 프로그래밍 교육"
대조군 A: "코딩으로 꿈을 키우세요"
실험군 B: "코딩으로 미래를

### 3.4 D+E등급 테스트 (목적형 소매 + 인프라)

In [10]:
# D+E등급 A/B 테스트 실행
de_grade_results = run_ab_test(DE_GRADE_TEST_CASES, "D+E등급 (목적형 소매 + 인프라)")


📊 D+E등급 (목적형 소매 + 인프라) A/B 테스트 결과


[1/30] D-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 타이어 전문점, 사계절 타이어 할인
   감지된 업종: s4_neat_variety
   추천 톤 적용: warm
   톤: warm, 최대 20자
✅ 생성 완료: 안전하게 달리는 타이어 할인!
한글 입력: "타이어 전문점, 사계절 타이어 할인"
대조군 A: "사계절 타이어 대세 할인!"
실험군 B: "안전하게 달리는 타이어 할인!"
📏 길이 비교: A=14자, B=16자

[2/30] D-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 자동차 부품점, 순정 브레이크 패드
   감지된 업종: c3_technical
   추천 톤 적용: professional
   톤: professional, 최대 20자
✅ 생성 완료: 믿고 쓰는 순정 브레이크 패드
한글 입력: "자동차 부품점, 순정 브레이크 패드"
대조군 A: "안전의 시작, 순정 브레이크 패드"
실험군 B: "믿고 쓰는 순정 브레이크 패드"
📏 길이 비교: A=18자, B=16자

[3/30] D-1
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 철물점, 전동 공구 세트 특가
   감지된 업종: d1_equipment
   추천 톤 적용: practical
   톤: practical, 최대 20자
✅ 생성 완료: 전동 공구 당일 출고!
한글 입력: "철물점, 전동 공구 세트 특가"
대조군 A: "전동 공구 세트 특가!"
실험군 B: "전동 공구 당일 출고!"


## 4. 종합 분석 및 결과 저장

In [11]:
import pandas as pd
from datetime import datetime

# 모든 결과 합치기
all_results = s_grade_results + a_grade_results + bc_grade_results + de_grade_results

# DataFrame으로 변환
df = pd.DataFrame(all_results)

# 기본 통계
print("\n" + "="*100)
print("📊 전체 A/B 테스트 종합 결과")
print("="*100)

print(f"\n총 테스트 케이스: {len(all_results)}개")
print(f"\n대조군 A 평균 길이: {df['len_a'].mean():.1f}자")
print(f"실험군 B 평균 길이: {df['len_b'].mean():.1f}자")
print(f"\n대조군 A 길이 범위: {df['len_a'].min()} ~ {df['len_a'].max()}자")
print(f"실험군 B 길이 범위: {df['len_b'].min()} ~ {df['len_b'].max()}자")

# 그룹별 통계
print("\n" + "-"*50)
print("그룹별 평균 길이 비교")
print("-"*50)
group_stats = df.groupby('group').agg({
    'len_a': 'mean',
    'len_b': 'mean'
}).round(1)
group_stats.columns = ['대조군 A 평균', '실험군 B 평균']
print(group_stats)

# 결과 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"ab_test_ad_copy_results_{timestamp}.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n💾 결과 저장 완료: {output_path}")


📊 전체 A/B 테스트 종합 결과

총 테스트 케이스: 170개

대조군 A 평균 길이: 13.3자
실험군 B 평균 길이: 16.0자

대조군 A 길이 범위: 9 ~ 20자
실험군 B 길이 범위: 11 ~ 20자

--------------------------------------------------
그룹별 평균 길이 비교
--------------------------------------------------
       대조군 A 평균  실험군 B 평균
group                    
A-1        13.4      15.7
A-2        12.5      17.7
A-3        12.8      17.5
A-4        13.6      15.3
B-1        13.6      16.6
B-2        12.9      17.6
B-3        12.8      15.9
C-1        13.8      16.1
C-2        13.1      17.1
C-3        14.0      16.2
D-1        15.1      14.8
D-2        15.3      14.0
E-1        13.0      17.6
E-2        12.2      14.0
S-1        11.9      16.0
S-2        13.4      14.9
S-3        13.4      15.7
S-4        12.4      15.7

💾 결과 저장 완료: ab_test_ad_copy_results_20260122_123423.csv


## 5. 샘플 비교 (상세 분석용)

In [12]:
def display_comparison(results, n_samples=3):
    """
    A/B 비교 결과를 보기 좋게 출력
    """
    import random
    samples = random.sample(results, min(n_samples, len(results)))
    
    for i, sample in enumerate(samples, 1):
        print(f"\n{'='*100}")
        print(f"샘플 {i}: {sample['group']}")
        print(f"{'='*100}")
        print(f"\n📝 한글 입력:")
        print(f"   \"{sample['korean_input']}\"")
        
        print(f"\n🔵 대조군 A ({sample['len_a']}자):")
        print(f"   \"{sample['ad_copy_a']}\"")
        
        print(f"\n🟢 실험군 B ({sample['len_b']}자):")
        print(f"   \"{sample['ad_copy_b']}\"")

# 각 등급에서 샘플 추출
print("\n" + "#"*100)
print("# S등급 샘플 비교")
print("#"*100)
display_comparison(s_grade_results, 2)

print("\n" + "#"*100)
print("# A등급 샘플 비교")
print("#"*100)
display_comparison(a_grade_results, 2)

print("\n" + "#"*100)
print("# B+C등급 샘플 비교")
print("#"*100)
display_comparison(bc_grade_results, 2)

print("\n" + "#"*100)
print("# D+E등급 샘플 비교")
print("#"*100)
display_comparison(de_grade_results, 2)


####################################################################################################
# S등급 샘플 비교
####################################################################################################

샘플 1: S-3

📝 한글 입력:
   "베이커리 오픈, 갓 구운 소금빵"

🔵 대조군 A (16자):
   "따끈한 소금빵, 지금 오세요!"

🟢 실험군 B (15자):
   "달콤한 위로, 소금빵의 행복"

샘플 2: S-1

📝 한글 입력:
   "중국집 짜장면, 정통 손맛 전수"

🔵 대조군 A (10자):
   "정통 짜장면의 맛!"

🟢 실험군 B (18자):
   "갓 만든 짜장면, 지금 맛보세요!"

####################################################################################################
# A등급 샘플 비교
####################################################################################################

샘플 1: A-1

📝 한글 입력:
   "피부관리실, 맑고 깨끗한 피부 케어"

🔵 대조군 A (11자):
   "피부의 맑음을 되찾다"

🟢 실험군 B (13자):
   "전문가의 맞춤 피부 변신"

샘플 2: A-4

📝 한글 입력:
   "세탁소 서비스, 명품 가방 클리닝"

🔵 대조군 A (16자):
   "명품 가방, 새처럼 깨끗하게!"

🟢 실험군 B (16자):
   "명품 가방, 섬세하게 관리해요"

###################################################################################################

## 6. GPT 기반 광고 문구 품질 평가

### 6.1 평가 기준 정의 (업종별 분류 문서 기반)

In [13]:
# ============================================
# GPT 기반 광고 문구 품질 평가 시스템
# ============================================

# 등급별 평가 기준 (업종별_분류_및_광고전략.md 기반)
EVALUATION_CRITERIA = {
    "S": {
        "name": "S등급 - 감성 및 미각 자극",
        "criteria": """
## S등급 광고 문구 평가 기준 (감성 및 미각 자극)

### 핵심 원칙
- 이성적인 판단보다 '시각적/감각적 자극'에 의한 즉각적 반응 유도
- 광고 문구 한 줄이 고객의 식욕과 감성을 자극해야 함

### 세부 평가 항목 (각 20점, 총 100점)

1. **감각적 표현 (Sensory Appeal)** - 20점
   - S-1: 육즙, 겉바속촉, 지글지글, 갓 구운 등 식욕 자극 표현
   - S-2: 신선함, 산지직송, 프리미엄, 투명한 등 고급감 표현
   - S-3: 달콤한, 부드러운, 촉촉한 등 감성적 표현
   - S-4: 정갈한, 엄마 손맛, 정성 등 가정적 표현

2. **타겟 고객 적합성 (Target Fit)** - 20점
   - S-1: 20-40대 직장인, 회식/야식 상황
   - S-2: 30-50대, 특별한 날/접대 상황
   - S-3: 20-30대 여성, SNS 활동적
   - S-4: 30-60대, 바쁜 직장인/1인 가구

3. **브랜드 보이스 (Tone & Manner)** - 20점
   - S-1: 친근하고 감각적, 식욕 자극
   - S-2: 신뢰감 있고 고급스러운
   - S-3: 따뜻하고 감성적, 위로하는
   - S-4: 따뜻하고 믿음직한

4. **CTA 명확성 (Call-to-Action)** - 20점
   - 고객이 취해야 할 행동이 명확한가
   - 지금 주문, 방문, 전화 등 구체적 행동 유도

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어/키워드 포함 여부
"""
    },
    "A": {
        "name": "A등급 - 비주얼 변화 및 자기관리",
        "criteria": """
## A등급 광고 문구 평가 기준 (비주얼 변화 및 자기관리)

### 핵심 원칙
- '변화'와 '결과'를 언어로 표현하여 기대감 형성
- 전문성과 개인 맞춤 서비스 강조

### 세부 평가 항목 (각 20점, 총 100점)

1. **변화/결과 표현 (Transformation)** - 20점
   - A-1: 찰랑거리는 머릿결, 맑아진 피부, 아름다운 변화
   - A-2: 탄탄한 몸매, -10kg 성공, 건강한 변화
   - A-3: 나다운 스타일, 트렌디한, 세련된
   - A-4: 섬세한, 특별한, 유니크한

2. **타겟 고객 적합성 (Target Fit)** - 20점
   - A-1: 20-40대 여성, 변화 원함
   - A-2: 25-45대, 건강/체력 목표
   - A-3: 20-30대, 트렌드+실용적 직장인
   - A-4: 20-40대, 선물/특별한 케어

3. **브랜드 보이스 (Tone & Manner)** - 20점
   - A-1: 전문적이면서 친근한
   - A-2: 동기부여하고 격려하는
   - A-3: 친근하고 트렌디한
   - A-4: 섬세하고 친절한

4. **전문성/신뢰감 (Professionalism)** - 20점
   - 전문가, 경력, 맞춤 서비스 등 신뢰 요소
   - 1:1 상담, 개인 맞춤 등 차별화 요소

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어/키워드 포함 여부
"""
    },
    "BC": {
        "name": "B+C등급 - 공간 체험/신뢰 기반 서비스",
        "criteria": """
## B+C등급 광고 문구 평가 기준 (공간 체험 + 신뢰 기반 서비스)

### B등급 핵심 원칙
- '머무는 경험'과 '성취'를 언어로 표현
- 시설의 품질과 환경 강조

### C등급 핵심 원칙
- '실패 없는 선택'임을 증명하는 신뢰성 강조
- 전문 자격, 경력, 정확한 정보 전달

### 세부 평가 항목 (각 20점, 총 100점)

1. **핵심 가치 전달 (Core Value)** - 20점
   - B-1: 집중, 성장, 쾌적함, 체계적 관리
   - B-2: 휴식, 탈출, 재충전, 힐링
   - B-3: 활력, 시설 품질, 실력 향상
   - C등급: 전문성, 신뢰, 안심

2. **타겟 고객 적합성 (Target Fit)** - 20점
   - B-1: 학부모/성인 학습자
   - B-2: 30-50대 가족/커플
   - B-3: 30-60대 운동 관심층
   - C등급: 전문 서비스 필요 고객

3. **브랜드 보이스 (Tone & Manner)** - 20점
   - B등급: 전문적/활기찬/따뜻한
   - C등급: 믿음직하고 전문적인

4. **신뢰 요소 (Trust Elements)** - 20점
   - 경력, 자격, 시설, 실적 등 객관적 신뢰 요소
   - 전문가, 합격률, A/S 보증 등

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어/키워드 포함 여부
"""
    },
    "DE": {
        "name": "D+E등급 - 목적형 소매/인프라",
        "criteria": """
## D+E등급 광고 문구 평가 기준 (목적형 소매 + 인프라)

### D등급 핵심 원칙
- 견고함, 실용성, 기능성 강조
- 가격, 재고, 품질 정보 명확히 전달

### E등급 핵심 원칙
- 접근성, 편의성, 품위 강조
- 직관적이고 명확한 정보 전달

### 세부 평가 항목 (각 20점, 총 100점)

1. **핵심 정보 전달 (Key Information)** - 20점
   - D-1: 재고 보유, 당일 출고, 정품, 내구성
   - D-2: 최신 모델, 고사양, 할인, A/S
   - E-1: 위치, 가격, 24시간, 편의시설
   - E-2: 엄숙한, 정중한, 깨끗한, 세심한

2. **타겟 고객 적합성 (Target Fit)** - 20점
   - D-1: 30-60대 차주/공사 관계자
   - D-2: 20-50대 기기 구매 계획자
   - E-1: 모든 연령대 운전자
   - E-2: 장례 서비스 필요 고객

3. **브랜드 보이스 (Tone & Manner)** - 20점
   - D등급: 실용적이고 직관적인
   - E-1: 직관적이고 명확한
   - E-2: 차분하고 정중한

4. **실용성/혜택 (Practicality)** - 20점
   - 가격 혜택, 할인, 특가 등 실질적 이익
   - 편의성, 접근성 등 실용적 가치

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어/키워드 포함 여부
"""
    }
}

print("평가 기준 로드 완료!")

평가 기준 로드 완료!


### 6.2 GPT 평가 함수 정의

In [14]:
def evaluate_ad_copy_with_gpt(
    korean_input: str,
    ad_copy_a: str,
    ad_copy_b: str,
    group: str
) -> Dict:
    """
    GPT를 사용하여 두 광고 문구를 비교 평가

    Args:
        korean_input: 원본 한글 입력
        ad_copy_a: 대조군 A 광고 문구
        ad_copy_b: 실험군 B 광고 문구
        group: 그룹 코드 (S-1, A-2 등)

    Returns:
        Dict: 평가 결과
    """
    # 등급 결정
    if group.startswith("S"):
        grade = "S"
    elif group.startswith("A"):
        grade = "A"
    elif group.startswith("B") or group.startswith("C"):
        grade = "BC"
    else:
        grade = "DE"

    criteria = EVALUATION_CRITERIA[grade]["criteria"]
    grade_name = EVALUATION_CRITERIA[grade]["name"]

    system_prompt = f"""당신은 소상공인 광고 문구 품질을 평가하는 전문가입니다.
광고 문구의 효과성과 업종 적합성을 평가합니다.

{criteria}

## 평가 방법
1. 각 항목별로 0-20점 척도로 평가
2. 두 광고 문구(A와 B)를 동일한 기준으로 평가
3. 어떤 광고 문구가 해당 업종에 더 효과적인지 판단
4. 반드시 JSON 형식으로 출력

## 출력 형식 (JSON)
{{
    "grade": "{grade_name}",
    "evaluation_a": {{
        "sensory_or_value": <0-20>,
        "target_fit": <0-20>,
        "tone_manner": <0-20>,
        "cta_or_trust": <0-20>,
        "industry_specific": <0-20>,
        "total": <0-100>,
        "strengths": "강점 설명",
        "weaknesses": "약점 설명"
    }},
    "evaluation_b": {{
        "sensory_or_value": <0-20>,
        "target_fit": <0-20>,
        "tone_manner": <0-20>,
        "cta_or_trust": <0-20>,
        "industry_specific": <0-20>,
        "total": <0-100>,
        "strengths": "강점 설명",
        "weaknesses": "약점 설명"
    }},
    "winner": "A" 또는 "B" 또는 "TIE",
    "winner_reason": "선택 이유 (2-3문장)",
    "detailed_comparison": "두 광고 문구의 상세 비교 분석 (3-4문장)"
}}
"""

    user_prompt = f"""## 평가 대상

### 원본 한글 입력 (광고 요청)
"{korean_input}"

### 업종 그룹
{group}

### 대조군 A 광고 문구
"{ad_copy_a}"

### 실험군 B 광고 문구
"{ad_copy_b}"

위 두 광고 문구를 평가 기준에 따라 비교 분석하고, 어떤 광고 문구가 해당 업종에 더 효과적인지 판단해주세요.
반드시 JSON 형식으로만 응답해주세요."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # 더 정확한 평가를 위해 gpt-4o 사용
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,  # 일관된 평가를 위해 낮은 temperature
            response_format={"type": "json_object"}
        )

        result = json.loads(response.choices[0].message.content)
        return result

    except Exception as e:
        return {"error": str(e)}


def evaluate_existing_results(existing_results: List[Dict], group_name: str, sample_size: int = None):
    """
    이미 생성된 A/B 테스트 결과를 사용하여 GPT 평가만 수행
    
    Args:
        existing_results: 섹션 3에서 생성된 A/B 테스트 결과 리스트
        group_name: 그룹명
        sample_size: 평가할 샘플 수 (None이면 전체 평가)
    
    Returns:
        평가 결과 리스트, 승패 통계
    """
    import random
    import time
    
    print(f"\n{'='*100}")
    print(f"📊 {group_name} GPT 평가 (기존 A/B 결과 활용)")
    print(f"{'='*100}\n")
    
    # 샘플 선택 (None이면 전체)
    if sample_size and sample_size < len(existing_results):
        samples = random.sample(existing_results, sample_size)
        print(f"📌 전체 {len(existing_results)}개 중 {sample_size}개 샘플 평가")
    else:
        samples = existing_results
        print(f"📌 전체 {len(existing_results)}개 평가")
    
    results = []
    wins = {"A": 0, "B": 0, "TIE": 0}
    total_scores = {"A": [], "B": []}
    
    for i, case in enumerate(samples, 1):
        group = case.get("group", "")
        korean_input = case.get("korean_input", "")
        ad_copy_a = case.get("ad_copy_a", "")
        ad_copy_b = case.get("ad_copy_b", "")
        
        print(f"\n[{i}/{len(samples)}] {group}")
        print(f"-" * 80)
        print(f'한글 입력: "{korean_input}"')
        print(f'대조군 A: "{ad_copy_a}"')
        print(f'실험군 B: "{ad_copy_b}"')
        
        # GPT 평가
        print("\n🔍 GPT 평가 중...")
        evaluation = evaluate_ad_copy_with_gpt(korean_input, ad_copy_a, ad_copy_b, group)
        
        if "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            winner = evaluation.get("winner", "TIE")
            
            print(f"\n📊 평가 결과:")
            print(f"   대조군 A 점수: {eval_a.get('total', 'N/A')}점")
            print(f"   실험군 B 점수: {eval_b.get('total', 'N/A')}점")
            print(f"   🏆 승자: {'대조군 A' if winner == 'A' else '실험군 B' if winner == 'B' else '무승부'}")
            print(f"   📝 이유: {evaluation.get('winner_reason', 'N/A')}")
            
            wins[winner] += 1
            if eval_a.get('total'):
                total_scores["A"].append(eval_a.get('total'))
            if eval_b.get('total'):
                total_scores["B"].append(eval_b.get('total'))
            
            results.append({
                "group": group,
                "korean_input": korean_input,
                "ad_copy_a": ad_copy_a,
                "ad_copy_b": ad_copy_b,
                "len_a": case.get("len_a", len(ad_copy_a)),
                "len_b": case.get("len_b", len(ad_copy_b)),
                "score_a": eval_a.get('total'),
                "score_b": eval_b.get('total'),
                "winner": winner,
                "evaluation": evaluation
            })
        else:
            print(f"   ❌ 평가 오류: {evaluation['error']}")
            # 오류가 나도 결과에 포함 (평가 없이)
            results.append({
                "group": group,
                "korean_input": korean_input,
                "ad_copy_a": ad_copy_a,
                "ad_copy_b": ad_copy_b,
                "len_a": case.get("len_a", len(ad_copy_a)),
                "len_b": case.get("len_b", len(ad_copy_b)),
                "score_a": None,
                "score_b": None,
                "winner": None,
                "evaluation": evaluation
            })
        
        # API 레이트 리밋 방지를 위한 딜레이
        if i < len(samples):
            time.sleep(0.5)
    
    # 요약 통계
    valid_results = [r for r in results if r.get("winner")]
    
    print(f"\n\n{'='*100}")
    print(f"📈 {group_name} 평가 요약")
    print(f"{'='*100}")
    print(f"평가 케이스 수: {len(valid_results)}/{len(samples)}")
    
    if valid_results:
        print(f"\n🏆 승패 현황:")
        print(f"   대조군 A 승: {wins['A']}회 ({wins['A']/len(valid_results)*100:.1f}%)")
        print(f"   실험군 B 승: {wins['B']}회 ({wins['B']/len(valid_results)*100:.1f}%)")
        print(f"   무승부: {wins['TIE']}회 ({wins['TIE']/len(valid_results)*100:.1f}%)")
    
    if total_scores["A"] and total_scores["B"]:
        print(f"\n📊 평균 점수:")
        print(f"   대조군 A: {sum(total_scores['A'])/len(total_scores['A']):.1f}점")
        print(f"   실험군 B: {sum(total_scores['B'])/len(total_scores['B']):.1f}점")
    
    return results, wins


# 기존 run_ab_test_with_evaluation 함수는 유지 (필요시 사용)
def run_ab_test_with_evaluation(test_cases: List[Dict], group_name: str, sample_size: int = 5):
    """
    A/B 테스트 실행 + GPT 평가 (새로 생성하면서 평가)
    - 주의: 이 함수는 A/B 결과를 새로 생성합니다.
    - 기존 결과를 활용하려면 evaluate_existing_results()를 사용하세요.
    """
    import random
    
    print(f"\n{'='*100}")
    print(f"📊 {group_name} A/B 테스트 + GPT 평가 (새로 생성)")
    print(f"{'='*100}\n")
    
    samples = random.sample(test_cases, min(sample_size, len(test_cases)))
    
    results = []
    wins = {"A": 0, "B": 0, "TIE": 0}
    total_scores = {"A": [], "B": []}
    
    for i, case in enumerate(samples, 1):
        group = case.get("group", "")
        korean_input = case.get("input", "")
        
        print(f"\n[{i}/{len(samples)}] {group}")
        print(f"-" * 80)
        print(f'한글 입력: "{korean_input}"')
        
        # A/B 광고 문구 생성
        ad_copy_a = generate_ad_copy_control_a(korean_input)
        ad_copy_b = generate_ad_copy_experiment_b(korean_input)
        
        print(f'\n대조군 A: "{ad_copy_a}"')
        print(f'실험군 B: "{ad_copy_b}"')
        
        # GPT 평가
        print("\n🔍 GPT 평가 중...")
        evaluation = evaluate_ad_copy_with_gpt(korean_input, ad_copy_a, ad_copy_b, group)
        
        if "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            winner = evaluation.get("winner", "TIE")
            
            print(f"\n📊 평가 결과:")
            print(f"   대조군 A 점수: {eval_a.get('total', 'N/A')}점")
            print(f"   실험군 B 점수: {eval_b.get('total', 'N/A')}점")
            print(f"   🏆 승자: {'대조군 A' if winner == 'A' else '실험군 B' if winner == 'B' else '무승부'}")
            
            wins[winner] += 1
            if eval_a.get('total'):
                total_scores["A"].append(eval_a.get('total'))
            if eval_b.get('total'):
                total_scores["B"].append(eval_b.get('total'))
            
            results.append({
                "group": group,
                "korean_input": korean_input,
                "ad_copy_a": ad_copy_a,
                "ad_copy_b": ad_copy_b,
                "score_a": eval_a.get('total'),
                "score_b": eval_b.get('total'),
                "winner": winner,
                "evaluation": evaluation
            })
        else:
            print(f"   ❌ 평가 오류: {evaluation['error']}")
    
    return results, wins

### 6.3 S등급 GPT 평가 실행

In [15]:
# S등급 GPT 평가 - 섹션 3에서 생성된 s_grade_results 활용
# sample_size=None 으로 설정하면 전체 40개 평가 (API 비용 주의)
# sample_size=10 으로 설정하면 10개만 샘플링하여 평가

s_eval_results, s_wins = evaluate_existing_results(
    s_grade_results,  # 섹션 3에서 생성된 결과 사용
    "S등급 (감성 및 미각 자극)",
    sample_size=None  # 전체 평가 (또는 원하는 샘플 수로 변경)
)


📊 S등급 (감성 및 미각 자극) GPT 평가 (기존 A/B 결과 활용)

📌 전체 40개 평가

[1/40] S-1
--------------------------------------------------------------------------------
한글 입력: "삼겹살 맛집, 숯불에 직접 굽는 고기"
대조군 A: "숯불 삼겹살의 진수!"
실험군 B: "숯불 삼겹살 지금 주문!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 84점
   실험군 B 점수: 88점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 즉각적인 행동을 유도하는 명확한 CTA를 포함하고 있어, 고객의 즉각적인 반응을 유도하는 데 더 효과적입니다.

[2/40] S-1
--------------------------------------------------------------------------------
한글 입력: "치킨 전문점 야식 배달, 갓 튀긴 바삭함"
대조군 A: "바삭한 치킨, 지금 배달!"
실험군 B: "갓 튀긴 바삭한 치킨"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 90점
   실험군 B 점수: 91점
   🏆 승자: 실험군 B
   📝 이유: 실험군 B 광고 문구는 갓 튀긴 치킨의 신선함과 바삭함을 강조하여 감각적 자극이 뛰어나다. 이는 고객의 식욕을 더 강하게 자극할 수 있다.

[3/40] S-1
--------------------------------------------------------------------------------
한글 입력: "피자 신메뉴 출시, 치즈가 듬뿍"
대조군 A: "치즈 폭발 피자 등장!"
실험군 B: "치즈 한가득, 지금 주문!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 86점
   실험군 B 점수: 87점
   🏆 승자: 실험군 B
   📝 이유: B 광고 문구는 치즈의 풍부함을 강조하면서도 명확한 주문 유도를 포함하여 고객의

### 6.4 A등급 GPT 평가 실행

In [16]:
# A등급 GPT 평가 - 섹션 3에서 생성된 a_grade_results 활용
a_eval_results, a_wins = evaluate_existing_results(
    a_grade_results,  # 섹션 3에서 생성된 결과 사용
    "A등급 (비주얼 변화 및 자기관리)",
    sample_size=None  # 전체 평가
)


📊 A등급 (비주얼 변화 및 자기관리) GPT 평가 (기존 A/B 결과 활용)

📌 전체 40개 평가

[1/40] A-1
--------------------------------------------------------------------------------
한글 입력: "미용실 봄 스타일, 웨이브 펌 이벤트"
대조군 A: "봄, 웨이브로 변신하자!"
실험군 B: "봄, 나만의 웨이브로 변신!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 87점
   실험군 B 점수: 91점
   🏆 승자: 실험군 B
   📝 이유: B 광고 문구는 개인 맞춤형 스타일을 강조하여 고객의 특별한 변화를 기대하게 만듭니다. 이는 타겟 고객에게 더 큰 매력을 줄 수 있습니다.

[2/40] A-1
--------------------------------------------------------------------------------
한글 입력: "피부관리실, 맑고 깨끗한 피부 케어"
대조군 A: "피부의 맑음을 되찾다"
실험군 B: "전문가의 맞춤 피부 변신"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 84점
   실험군 B 점수: 93점
   🏆 승자: 실험군 B
   📝 이유: 실험군 B는 전문가의 맞춤 서비스를 강조하여 신뢰감을 주고, 개인 맞춤형 변화를 제시하여 고객의 기대감을 높입니다. 이는 타겟 고객의 니즈에 더 적합하게 작용합니다.

[3/40] A-1
--------------------------------------------------------------------------------
한글 입력: "네일샵 신상 디자인, 봄꽃 네일 아트"
대조군 A: "봄꽃 네일로 새로움!"
실험군 B: "봄꽃 네일로 변신해요!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 87점
   실험군 B 점수: 90점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 '변신'이라는 단어

### 6.5 B+C등급 GPT 평가 실행

In [17]:
# B+C등급 GPT 평가 - 섹션 3에서 생성된 bc_grade_results 활용
bc_eval_results, bc_wins = evaluate_existing_results(
    bc_grade_results,  # 섹션 3에서 생성된 결과 사용
    "B+C등급 (공간 체험/교육 + 신뢰 기반 서비스)",
    sample_size=None  # 전체 평가
)


📊 B+C등급 (공간 체험/교육 + 신뢰 기반 서비스) GPT 평가 (기존 A/B 결과 활용)

📌 전체 60개 평가

[1/60] B-1
--------------------------------------------------------------------------------
한글 입력: "입시학원, 소수정예 수학 클래스"
대조군 A: "수학, 소수로 깊이 있게!"
실험군 B: "소수정예로 성적 UP! 지금 상담 신"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 86점
   실험군 B 점수: 91점
   🏆 승자: 실험군 B
   📝 이유: 실험군 B 광고 문구는 소수 정예와 성적 향상이라는 명확한 가치를 전달하며, 즉각적인 상담을 유도하는 행동 촉구가 효과적입니다.

[2/60] B-1
--------------------------------------------------------------------------------
한글 입력: "외국어 학원, 원어민 영어 회화"
대조군 A: "원어민과의 자유로운 영어!"
실험군 B: "원어민과 함께하는 영어!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 78점
   실험군 B 점수: 77점
   🏆 승자: 대조군 A
   📝 이유: 광고 문구 A는 '자유로운 영어'라는 표현을 통해 학습자에게 더 매력적이고 개방적인 학습 환경을 제공할 것이라는 인상을 줍니다.

[3/60] B-1
--------------------------------------------------------------------------------
한글 입력: "코딩 학원, 어린이 프로그래밍 교육"
대조군 A: "코딩으로 꿈을 키우세요"
실험군 B: "코딩으로 미래를 여는 첫걸음!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 75점
   실험군 B 점수: 81점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 '미래를 여는 첫걸음'이라는 표현으로

### 6.6 D+E등급 GPT 평가 실행

In [18]:
# D+E등급 GPT 평가 - 섹션 3에서 생성된 de_grade_results 활용
de_eval_results, de_wins = evaluate_existing_results(
    de_grade_results,  # 섹션 3에서 생성된 결과 사용
    "D+E등급 (목적형 소매 + 인프라)",
    sample_size=None  # 전체 평가
)


📊 D+E등급 (목적형 소매 + 인프라) GPT 평가 (기존 A/B 결과 활용)

📌 전체 30개 평가

[1/30] D-1
--------------------------------------------------------------------------------
한글 입력: "타이어 전문점, 사계절 타이어 할인"
대조군 A: "사계절 타이어 대세 할인!"
실험군 B: "안전하게 달리는 타이어 할인!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 81점
   실험군 B 점수: 85점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 안전성을 강조하여 타이어의 본질적인 기능을 잘 전달하고 있으며, 할인 혜택도 함께 제공하여 실용성을 높임.

[2/30] D-1
--------------------------------------------------------------------------------
한글 입력: "자동차 부품점, 순정 브레이크 패드"
대조군 A: "안전의 시작, 순정 브레이크 패드"
실험군 B: "믿고 쓰는 순정 브레이크 패드"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 82점
   실험군 B 점수: 86점
   🏆 승자: 실험군 B
   📝 이유: 실험군 B 광고 문구는 신뢰성을 강조하여 고객의 구매 결정을 유도하는 데 효과적입니다. 이는 자동차 부품점에서 중요한 요소로 작용할 수 있습니다.

[3/30] D-1
--------------------------------------------------------------------------------
한글 입력: "철물점, 전동 공구 세트 특가"
대조군 A: "전동 공구 세트 특가!"
실험군 B: "전동 공구 당일 출고!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 81점
   실험군 B 점수: 86점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 당일 출고라는 즉시성을 강조하여 실용적

### 6.7 전체 GPT 평가 종합 결과

In [19]:
# 전체 결과 종합
all_eval_results = s_eval_results + a_eval_results + bc_eval_results + de_eval_results
all_wins = {
    "A": s_wins["A"] + a_wins["A"] + bc_wins["A"] + de_wins["A"],
    "B": s_wins["B"] + a_wins["B"] + bc_wins["B"] + de_wins["B"],
    "TIE": s_wins["TIE"] + a_wins["TIE"] + bc_wins["TIE"] + de_wins["TIE"]
}

total_evaluated = len(all_eval_results)

print("\n" + "="*100)
print("🏆 전체 GPT 평가 종합 결과")
print("="*100)

print(f"\n총 평가 케이스: {total_evaluated}개")
print(f"\n📊 전체 승패 현황:")
print(f"   대조군 A 승: {all_wins['A']}회 ({all_wins['A']/total_evaluated*100:.1f}%)")
print(f"   실험군 B 승: {all_wins['B']}회 ({all_wins['B']/total_evaluated*100:.1f}%)")
print(f"   무승부: {all_wins['TIE']}회 ({all_wins['TIE']/total_evaluated*100:.1f}%)")

# 등급별 결과 테이블
print("\n" + "-"*50)
print("등급별 승패 현황")
print("-"*50)
grade_results = pd.DataFrame({
    "등급": ["S등급", "A등급", "B+C등급", "D+E등급", "전체"],
    "대조군 A 승": [s_wins["A"], a_wins["A"], bc_wins["A"], de_wins["A"], all_wins["A"]],
    "실험군 B 승": [s_wins["B"], a_wins["B"], bc_wins["B"], de_wins["B"], all_wins["B"]],
    "무승부": [s_wins["TIE"], a_wins["TIE"], bc_wins["TIE"], de_wins["TIE"], all_wins["TIE"]]
})
print(grade_results.to_string(index=False))

# 평균 점수 계산
all_scores_a = [r["score_a"] for r in all_eval_results if r.get("score_a")]
all_scores_b = [r["score_b"] for r in all_eval_results if r.get("score_b")]

if all_scores_a and all_scores_b:
    print(f"\n📈 전체 평균 점수:")
    print(f"   대조군 A: {sum(all_scores_a)/len(all_scores_a):.1f}점")
    print(f"   실험군 B: {sum(all_scores_b)/len(all_scores_b):.1f}점")
    print(f"   점수 차이: {sum(all_scores_b)/len(all_scores_b) - sum(all_scores_a)/len(all_scores_a):+.1f}점")

# 결과 저장
eval_df = pd.DataFrame([{
    "group": r["group"],
    "korean_input": r["korean_input"],
    "score_a": r["score_a"],
    "score_b": r["score_b"],
    "winner": r["winner"],
    "ad_copy_a": r["ad_copy_a"],
    "ad_copy_b": r["ad_copy_b"]
} for r in all_eval_results])

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
eval_output_path = f"ab_test_ad_copy_gpt_evaluation_{timestamp}.csv"
eval_df.to_csv(eval_output_path, index=False, encoding='utf-8-sig')
print(f"\n💾 GPT 평가 결과 저장 완료: {eval_output_path}")


🏆 전체 GPT 평가 종합 결과

총 평가 케이스: 170개

📊 전체 승패 현황:
   대조군 A 승: 19회 (11.2%)
   실험군 B 승: 146회 (85.9%)
   무승부: 5회 (2.9%)

--------------------------------------------------
등급별 승패 현황
--------------------------------------------------
   등급  대조군 A 승  실험군 B 승  무승부
  S등급        2       38    0
  A등급        2       36    2
B+C등급        9       50    1
D+E등급        6       22    2
   전체       19      146    5

📈 전체 평균 점수:
   대조군 A: 77.9점
   실험군 B: 83.7점
   점수 차이: +5.8점

💾 GPT 평가 결과 저장 완료: ab_test_ad_copy_gpt_evaluation_20260122_125728.csv


## 7. 상세 평가 분석 (샘플)

In [20]:
def display_detailed_evaluation(results, n_samples=2):
    """
    상세 평가 결과를 보기 좋게 출력
    """
    import random
    samples = random.sample(results, min(n_samples, len(results)))

    for i, sample in enumerate(samples, 1):
        evaluation = sample.get("evaluation", {})
        eval_a = evaluation.get("evaluation_a", {})
        eval_b = evaluation.get("evaluation_b", {})

        print(f"\n{'='*100}")
        print(f"📋 상세 평가 샘플 {i}: {sample['group']}")
        print(f"{'='*100}")

        print(f"\n📝 한글 입력: \"{sample['korean_input']}\"")

        print(f"\n{'─'*50}")
        print("🔵 대조군 A")
        print(f"{'─'*50}")
        print(f"광고 문구: \"{sample['ad_copy_a']}\"")
        print(f"\n점수 상세:")
        print(f"  - 감각/가치 표현: {eval_a.get('sensory_or_value', 'N/A')}/20")
        print(f"  - 타겟 적합성: {eval_a.get('target_fit', 'N/A')}/20")
        print(f"  - 톤앤매너: {eval_a.get('tone_manner', 'N/A')}/20")
        print(f"  - CTA/신뢰: {eval_a.get('cta_or_trust', 'N/A')}/20")
        print(f"  - 업종 특화: {eval_a.get('industry_specific', 'N/A')}/20")
        print(f"  📊 총점: {eval_a.get('total', 'N/A')}/100")
        print(f"\n강점: {eval_a.get('strengths', 'N/A')}")
        print(f"약점: {eval_a.get('weaknesses', 'N/A')}")

        print(f"\n{'─'*50}")
        print("🟢 실험군 B")
        print(f"{'─'*50}")
        print(f"광고 문구: \"{sample['ad_copy_b']}\"")
        print(f"\n점수 상세:")
        print(f"  - 감각/가치 표현: {eval_b.get('sensory_or_value', 'N/A')}/20")
        print(f"  - 타겟 적합성: {eval_b.get('target_fit', 'N/A')}/20")
        print(f"  - 톤앤매너: {eval_b.get('tone_manner', 'N/A')}/20")
        print(f"  - CTA/신뢰: {eval_b.get('cta_or_trust', 'N/A')}/20")
        print(f"  - 업종 특화: {eval_b.get('industry_specific', 'N/A')}/20")
        print(f"  📊 총점: {eval_b.get('total', 'N/A')}/100")
        print(f"\n강점: {eval_b.get('strengths', 'N/A')}")
        print(f"약점: {eval_b.get('weaknesses', 'N/A')}")

        print(f"\n{'─'*50}")
        print("🏆 평가 결론")
        print(f"{'─'*50}")
        winner = evaluation.get('winner', 'N/A')
        print(f"승자: {'대조군 A' if winner == 'A' else '실험군 B' if winner == 'B' else '무승부'}")
        print(f"이유: {evaluation.get('winner_reason', 'N/A')}")
        print(f"\n상세 비교: {evaluation.get('detailed_comparison', 'N/A')}")

# 전체 평가 결과에서 샘플 출력
if all_eval_results:
    print("\n" + "#"*100)
    print("# 상세 평가 분석 샘플")
    print("#"*100)
    display_detailed_evaluation(all_eval_results, 3)


####################################################################################################
# 상세 평가 분석 샘플
####################################################################################################

📋 상세 평가 샘플 1: B-3

📝 한글 입력: "배드민턴장, 동호회 대관 안내"

──────────────────────────────────────────────────
🔵 대조군 A
──────────────────────────────────────────────────
광고 문구: "함께 즐기는 배드민턴!"

점수 상세:
  - 감각/가치 표현: 15/20
  - 타겟 적합성: 18/20
  - 톤앤매너: 16/20
  - CTA/신뢰: 12/20
  - 업종 특화: 14/20
  📊 총점: 75/100

강점: 간결하고 직관적으로 배드민턴의 즐거움을 강조하여 타겟 고객에게 친근하게 다가감.
약점: 구체적인 예약이나 시설 관련 정보가 부족하여 신뢰 요소가 약함.

──────────────────────────────────────────────────
🟢 실험군 B
──────────────────────────────────────────────────
광고 문구: "함께하는 즐거움, 배드민턴 예약하세요"

점수 상세:
  - 감각/가치 표현: 16/20
  - 타겟 적합성: 18/20
  - 톤앤매너: 17/20
  - CTA/신뢰: 15/20
  - 업종 특화: 15/20
  📊 총점: 81/100

강점: 즐거움과 함께 예약을 유도하는 명확한 Call to Action을 포함하여 고객 참여를 촉진함.
약점: 시설의 품질이나 전문성에 대한 구체적인 정보가 부족함.

──────────────────────────────────────────────────
🏆

## 8. 결론

### 실험 설계

| 항목 | 대조군 A | 실험군 B |
|------|---------|----------|
| 방식 | 일반 OpenAI API | 업종별 특화 시스템 |
| 광고 문구 구조 | 단순 번역 + 일반 지시문 | 18개 하위 그룹별 최적화 |
| 톤앤매너 | 고정 | 업종별 자동 선택 |
| AIDA 프레임워크 | X | O |
| 타겟 고객 반영 | X | O |
| 업종별 키워드 | X | O (YAML 기반) |

### GPT 평가 기준 (업종별_분류_및_광고전략.md 기반)

| 평가 항목 | 배점 | 설명 |
|----------|------|------|
| 감각/가치 표현 | 20점 | 업종별 핵심 감각적/가치 표현 |
| 타겟 적합성 | 20점 | 타겟 고객에 맞는 메시지 |
| 톤앤매너 | 20점 | 업종에 맞는 브랜드 보이스 |
| CTA/신뢰 | 20점 | 행동 유도 또는 신뢰 요소 |
| 업종 특화 키워드 | 20점 | 업종별 전문 용어 포함 |
| **총점** | **100점** | |

### 기대 효과

1. **업종별 최적화**: 실험군 B는 각 업종의 특성을 반영한 톤앤매너와 키워드를 자동 적용
2. **타겟 맞춤**: 업종별 타겟 고객에 최적화된 메시지 전달
3. **전문성**: 업종별 전문 용어와 광고 문법 활용
4. **객관적 검증**: GPT 기반 자동 평가로 품질 차이를 정량적으로 측정

## 9. B 실험군 패배/무승부 케이스 분석

실험군 B(업종별 특화 시스템)가 대조군 A(일반 OpenAI API)에게 패배하거나 무승부인 케이스를 분석합니다.
이 케이스들은 `industries.yaml` 템플릿 개선의 핵심 데이터가 됩니다.

In [ ]:
# ============================================
# B 실험군 패배/무승부 케이스 수집 및 분석
# ============================================

def collect_b_loss_or_tie_cases(all_eval_results: List[Dict]) -> List[Dict]:
    """
    B 실험군이 패배하거나 무승부인 케이스 수집
    
    Args:
        all_eval_results: 전체 GPT 평가 결과 리스트
    
    Returns:
        B가 지거나 비긴 케이스 리스트
    """
    b_loss_or_tie = []
    
    for result in all_eval_results:
        winner = result.get("winner")
        if winner in ["A", "TIE"]:  # A가 이기거나 무승부
            b_loss_or_tie.append(result)
    
    return b_loss_or_tie


def display_b_loss_cases(cases: List[Dict], title: str = "B 실험군 패배/무승부 케이스"):
    """
    B 실험군 패배/무승부 케이스를 상세히 출력
    """
    print(f"\n{'='*120}")
    print(f"📊 {title}")
    print(f"{'='*120}")
    print(f"총 {len(cases)}개 케이스 (A 승리 또는 무승부)\n")
    
    # 승패별 분류
    a_wins = [c for c in cases if c.get("winner") == "A"]
    ties = [c for c in cases if c.get("winner") == "TIE"]
    
    print(f"  - A 승리: {len(a_wins)}개")
    print(f"  - 무승부: {len(ties)}개")
    
    # 그룹별 분류
    group_counts = {}
    for case in cases:
        group = case.get("group", "Unknown")
        group_counts[group] = group_counts.get(group, 0) + 1
    
    print(f"\n📌 그룹별 분포:")
    for group, count in sorted(group_counts.items()):
        print(f"  - {group}: {count}개")
    
    print(f"\n{'='*120}")
    print("상세 케이스 분석")
    print(f"{'='*120}\n")
    
    for i, case in enumerate(cases, 1):
        group = case.get("group", "Unknown")
        korean_input = case.get("korean_input", "N/A")
        ad_copy_a = case.get("ad_copy_a", "N/A")
        ad_copy_b = case.get("ad_copy_b", "N/A")
        score_a = case.get("score_a", "N/A")
        score_b = case.get("score_b", "N/A")
        winner = case.get("winner", "N/A")
        evaluation = case.get("evaluation", {})
        
        winner_label = "🏆 A 승리" if winner == "A" else "🤝 무승부"
        
        print(f"\n[{i}/{len(cases)}] {group} - {winner_label}")
        print(f"-" * 100)
        
        # 입력 및 결과
        print(f"\n📝 한글 입력:")
        print(f"   \"{korean_input}\"")
        
        print(f"\n🅰️ 대조군 A 광고 문구 (점수: {score_a}점):")
        print(f"   \"{ad_copy_a}\"")
        
        print(f"\n🅱️ 실험군 B 광고 문구 (점수: {score_b}점):")
        print(f"   \"{ad_copy_b}\"")
        
        # GPT 평가 상세 내용
        if evaluation and "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            
            print(f"\n📊 GPT 평가 상세:")
            print(f"   [대조군 A]")
            print(f"   - 감성/가치 표현: {eval_a.get('sensory_or_value', 'N/A')}점")
            print(f"   - 타겟 적합성: {eval_a.get('target_fit', 'N/A')}점")
            print(f"   - 톤앤매너: {eval_a.get('tone_manner', 'N/A')}점")
            print(f"   - CTA/신뢰: {eval_a.get('cta_or_trust', 'N/A')}점")
            print(f"   - 업종 특화: {eval_a.get('industry_specific', 'N/A')}점")
            print(f"   - 강점: {eval_a.get('strengths', 'N/A')}")
            print(f"   - 약점: {eval_a.get('weaknesses', 'N/A')}")
            
            print(f"\n   [실험군 B]")
            print(f"   - 감성/가치 표현: {eval_b.get('sensory_or_value', 'N/A')}점")
            print(f"   - 타겟 적합성: {eval_b.get('target_fit', 'N/A')}점")
            print(f"   - 톤앤매너: {eval_b.get('tone_manner', 'N/A')}점")
            print(f"   - CTA/신뢰: {eval_b.get('cta_or_trust', 'N/A')}점")
            print(f"   - 업종 특화: {eval_b.get('industry_specific', 'N/A')}점")
            print(f"   - 강점: {eval_b.get('strengths', 'N/A')}")
            print(f"   - 약점: {eval_b.get('weaknesses', 'N/A')}")
            
            print(f"\n   📝 승패 이유: {evaluation.get('winner_reason', 'N/A')}")
            print(f"   📝 상세 비교: {evaluation.get('detailed_comparison', 'N/A')}")
        
        print(f"\n{'─'*100}")
    
    return cases


# B 패배/무승부 케이스 수집
b_loss_or_tie_cases = collect_b_loss_or_tie_cases(all_eval_results)

# 상세 출력
display_b_loss_cases(b_loss_or_tie_cases, "광고 문구 B 실험군 패배/무승부 케이스")

## 10. industries.yaml 개선사항 제안

B 실험군 패배/무승부 케이스를 분석하여 `src/generation/text_generation/config/industries.yaml` 파일의 개선사항을 GPT를 활용해 구체적으로 제안합니다.

In [ ]:
# ============================================
# industries.yaml 개선사항 제안 시스템
# ============================================

def generate_improvement_suggestions(
    b_loss_cases: List[Dict],
    yaml_file_path: str = "src/generation/text_generation/config/industries.yaml"
) -> Dict:
    """
    B 패배/무승부 케이스를 분석하여 industries.yaml 개선사항 제안
    
    Args:
        b_loss_cases: B가 지거나 비긴 케이스 리스트
        yaml_file_path: industries.yaml 파일 경로
    
    Returns:
        그룹별 개선사항 딕셔너리
    """
    if not b_loss_cases:
        print("분석할 B 패배/무승부 케이스가 없습니다.")
        return {}
    
    # 그룹별로 케이스 분류
    cases_by_group = {}
    for case in b_loss_cases:
        group = case.get("group", "Unknown")
        if group not in cases_by_group:
            cases_by_group[group] = []
        cases_by_group[group].append(case)
    
    print(f"\n{'='*120}")
    print(f"🔧 industries.yaml 개선사항 제안 (광고 문구)")
    print(f"{'='*120}")
    print(f"대상 파일: {yaml_file_path}")
    print(f"분석 대상: {len(b_loss_cases)}개 케이스 ({len(cases_by_group)}개 그룹)")
    print(f"{'='*120}\n")
    
    all_suggestions = {}
    
    for group, cases in cases_by_group.items():
        print(f"\n{'─'*100}")
        print(f"📌 {group} 그룹 분석 중... ({len(cases)}개 케이스)")
        print(f"{'─'*100}")
        
        # 케이스 요약 생성
        case_summaries = []
        for case in cases:
            summary = {
                "korean_input": case.get("korean_input", ""),
                "ad_copy_a": case.get("ad_copy_a", ""),
                "ad_copy_b": case.get("ad_copy_b", ""),
                "score_a": case.get("score_a"),
                "score_b": case.get("score_b"),
                "winner": case.get("winner"),
                "evaluation": {
                    "a_strengths": case.get("evaluation", {}).get("evaluation_a", {}).get("strengths", ""),
                    "a_weaknesses": case.get("evaluation", {}).get("evaluation_a", {}).get("weaknesses", ""),
                    "b_strengths": case.get("evaluation", {}).get("evaluation_b", {}).get("strengths", ""),
                    "b_weaknesses": case.get("evaluation", {}).get("evaluation_b", {}).get("weaknesses", ""),
                    "winner_reason": case.get("evaluation", {}).get("winner_reason", ""),
                    "detailed_comparison": case.get("evaluation", {}).get("detailed_comparison", "")
                }
            }
            case_summaries.append(summary)
        
        # GPT에게 개선사항 요청
        system_prompt = """당신은 소상공인 광고 문구 생성 시스템의 템플릿 전문가입니다.
A/B 테스트 결과를 분석하여 industries.yaml 파일의 개선사항을 제안합니다.

industries.yaml 파일은 다음과 같은 구조를 가집니다:
```yaml
industries:
  카테고리명:
    tone: "톤앤매너 (warm, professional, friendly 등)"
    keywords: [키워드1, 키워드2, ...]
    cta_style: "CTA 스타일"
    value_proposition: "가치 제안 방식"
    target_emotion: "타겟 감정"
    ad_template: "광고 문구 템플릿"
```

분석 결과를 바탕으로 다음을 제안해주세요:
1. 톤앤매너를 어떻게 개선해야 하는지
2. 어떤 키워드를 추가/수정해야 하는지
3. CTA 스타일을 어떻게 개선해야 하는지
4. 가치 제안 방식을 어떻게 개선해야 하는지
5. 타겟 감정을 어떻게 설정해야 하는지
6. 광고 문구 템플릿을 어떻게 개선해야 하는지

반드시 JSON 형식으로 출력하세요."""

        user_prompt = f"""## 분석 대상 그룹
{group}

## A/B 테스트 결과 (B 실험군이 패배하거나 무승부인 케이스)
{json.dumps(case_summaries, ensure_ascii=False, indent=2)}

## 요청사항
위 케이스들을 분석하여 실험군 B(업종별 특화 시스템)가 대조군 A(일반 시스템)보다 성능이 떨어진 원인을 파악하고,
industries.yaml 파일의 해당 그룹 설정을 어떻게 개선해야 하는지 구체적으로 제안해주세요.

특히:
1. 대조군 A가 더 좋았던 이유 분석
2. 실험군 B의 약점 분석
3. industries.yaml에서 개선해야 할 구체적인 항목과 값

출력 형식:
{{
    "group": "{group}",
    "analysis": {{
        "why_a_won": "A가 더 좋았던 이유",
        "b_weaknesses": "B의 약점 분석",
        "common_patterns": "공통적으로 발견된 패턴"
    }},
    "improvements": {{
        "tone": {{
            "current_issue": "현재 톤앤매너의 문제점",
            "suggestion": "개선된 톤앤매너",
            "reason": "변경 이유"
        }},
        "keywords": {{
            "add": ["추가할 키워드들"],
            "remove": ["제거할 키워드들"],
            "reason": "키워드 변경 이유"
        }},
        "cta_style": {{
            "current_issue": "현재 CTA 스타일의 문제점",
            "suggestion": "개선된 CTA 스타일",
            "reason": "변경 이유"
        }},
        "value_proposition": {{
            "current_issue": "현재 가치 제안의 문제점",
            "suggestion": "개선된 가치 제안 방식",
            "reason": "변경 이유"
        }},
        "target_emotion": {{
            "current_issue": "현재 타겟 감정의 문제점",
            "suggestion": "개선된 타겟 감정",
            "reason": "변경 이유"
        }},
        "ad_template": {{
            "current_issue": "현재 템플릿 문제점",
            "suggestion": "개선된 템플릿 예시",
            "reason": "변경 이유"
        }}
    }},
    "priority": "높음/중간/낮음",
    "summary": "전체 개선사항 요약 (2-3문장)"
}}"""

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.5,
                response_format={"type": "json_object"}
            )
            
            suggestion = json.loads(response.choices[0].message.content)
            all_suggestions[group] = suggestion
            
            # 결과 출력
            print(f"\n✅ {group} 분석 완료")
            print(f"\n📊 분석 결과:")
            print(f"   - A 승리 이유: {suggestion.get('analysis', {}).get('why_a_won', 'N/A')}")
            print(f"   - B 약점: {suggestion.get('analysis', {}).get('b_weaknesses', 'N/A')}")
            print(f"\n🔧 주요 개선사항:")
            
            improvements = suggestion.get('improvements', {})
            
            if improvements.get('tone', {}).get('suggestion'):
                print(f"   - 톤앤매너 개선: {improvements['tone']['suggestion']}")
            if improvements.get('keywords', {}).get('add'):
                print(f"   - 키워드 추가: {improvements['keywords']['add']}")
            if improvements.get('cta_style', {}).get('suggestion'):
                print(f"   - CTA 개선: {improvements['cta_style']['suggestion']}")
            if improvements.get('ad_template', {}).get('suggestion'):
                print(f"   - 템플릿 개선: {improvements['ad_template']['suggestion'][:150]}...")
            
            print(f"\n   📌 우선순위: {suggestion.get('priority', 'N/A')}")
            print(f"   📝 요약: {suggestion.get('summary', 'N/A')}")
            
        except Exception as e:
            print(f"   ❌ 분석 오류: {str(e)}")
            all_suggestions[group] = {"error": str(e)}
        
        # API 레이트 리밋 방지
        import time
        time.sleep(1)
    
    return all_suggestions


# 개선사항 생성
improvement_suggestions = generate_improvement_suggestions(b_loss_or_tie_cases)

In [ ]:
# ============================================
# 개선사항 종합 보고서 저장
# ============================================

def save_improvement_report(
    suggestions: Dict,
    b_loss_cases: List[Dict],
    output_path: str = "260122_광고문구_개선사항_보고서.json"
):
    """
    개선사항 보고서를 JSON 파일로 저장
    """
    report = {
        "report_type": "광고 문구 템플릿 개선사항",
        "target_file": "src/generation/text_generation/config/industries.yaml",
        "analysis_date": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
        "total_b_loss_cases": len(b_loss_cases),
        "groups_analyzed": len(suggestions),
        "suggestions_by_group": suggestions,
        "b_loss_cases_detail": b_loss_cases
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ 개선사항 보고서 저장 완료: {output_path}")
    return report


# 보고서 저장
improvement_report = save_improvement_report(
    improvement_suggestions,
    b_loss_or_tie_cases,
    "260122_광고문구_개선사항_보고서.json"
)

# 종합 요약 출력
print(f"\n{'='*120}")
print("📋 광고 문구 템플릿 개선사항 종합 요약")
print(f"{'='*120}")

priority_high = [g for g, s in improvement_suggestions.items() if s.get('priority') == '높음']
priority_medium = [g for g, s in improvement_suggestions.items() if s.get('priority') == '중간']
priority_low = [g for g, s in improvement_suggestions.items() if s.get('priority') == '낮음']

print(f"\n🔴 높은 우선순위 ({len(priority_high)}개 그룹): {priority_high}")
print(f"🟡 중간 우선순위 ({len(priority_medium)}개 그룹): {priority_medium}")
print(f"🟢 낮은 우선순위 ({len(priority_low)}개 그룹): {priority_low}")

print(f"\n📝 그룹별 핵심 개선사항:")
for group, suggestion in improvement_suggestions.items():
    if "error" not in suggestion:
        print(f"\n  [{group}]")
        print(f"  {suggestion.get('summary', 'N/A')}")